## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-26-15-07-26 +0000,nypost,Terrifying video shows California skier dangli...,https://nypost.com/2026/02/26/us-news/terrifyi...
1,2026-02-26-15-05-00 +0000,wsj,Dollar Trims Losses After Weekly Jobless Claim...,https://www.wsj.com/finance/currencies/asian-c...
2,2026-02-26-15-04-24 +0000,bbc,Hillary Clinton to appear before US House pane...,https://www.bbc.com/news/articles/cd6z3e1vdvzo...
3,2026-02-26-15-03-38 +0000,nyt,Kalshi Accuses a ‘MrBeast’ Employee of Insider...,https://www.nytimes.com/2026/02/25/business/ka...
4,2026-02-26-14-54-35 +0000,bbc,Spain to check Gibraltar arrivals under post-B...,https://www.bbc.com/news/articles/cgjz1x5e1xyo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2623/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
317,trump,61
91,iran,21
26,epstein,18
810,union,14
44,new,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
323,2026-02-25-18-29-00 +0000,wsj,"Casey Means, President Trump’s surgeon-general...",https://www.wsj.com/politics/policy/surgeon-ge...,129
62,2026-02-26-12-32-24 +0000,nypost,"New US, Iran talks underway in Geneva as Trump...",https://nypost.com/2026/02/26/us-news/new-us-i...,122
52,2026-02-26-13-00-00 +0000,wsj,Oil lobbyists have made the case to Trump offi...,https://www.wsj.com/politics/policy/big-oil-is...,117
202,2026-02-26-00-26-00 +0000,wsj,Here’s a look at whether the math works on som...,https://www.wsj.com/politics/policy/the-math-b...,115
322,2026-02-25-18-33-51 +0000,nypost,Dozen of world’s deadliest fighter jets arrive...,https://nypost.com/2026/02/25/world-news/world...,111


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
323,129,2026-02-25-18-29-00 +0000,wsj,"Casey Means, President Trump’s surgeon-general...",https://www.wsj.com/politics/policy/surgeon-ge...
251,73,2026-02-25-22-28-00 +0000,wsj,Spy agency blocks Congress from seeing Gabbard...,https://www.wsj.com/politics/policy/spy-agency...
62,61,2026-02-26-12-32-24 +0000,nypost,"New US, Iran talks underway in Geneva as Trump...",https://nypost.com/2026/02/26/us-news/new-us-i...
52,56,2026-02-26-13-00-00 +0000,wsj,Oil lobbyists have made the case to Trump offi...,https://www.wsj.com/politics/policy/big-oil-is...
172,48,2026-02-26-03-02-00 +0000,wsj,The Trump administration is pausing nearly $26...,https://www.wsj.com/politics/policy/vance-anno...
334,45,2026-02-25-17-17-41 +0000,latimes,FBI raids of LAUSD Supt. Alberto Carvalho's ho...,https://www.latimes.com/california/story/2026-...
374,44,2026-02-25-15-13-00 +0000,wsj,Didn’t catch all of President Trump’s State of...,https://www.wsj.com/politics/policy/trump-stat...
113,41,2026-02-26-10-39-25 +0000,nypost,Kim Jong Un calls South Korea ‘most hostile en...,https://nypost.com/2026/02/26/world-news/kim-j...
89,36,2026-02-26-11-39-56 +0000,nypost,Third relative of transgender dad shooter dies...,https://nypost.com/2026/02/26/us-news/transgen...
207,35,2026-02-26-00-11-00 +0000,wsj,Nvidia Beats Back Bubble Fears With Record $68...,https://www.wsj.com/business/earnings/nvidia-e...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
